<a href="https://colab.research.google.com/github/DHEERAJ264/Apache-Spark/blob/master/spark_transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


     |████████████████████████████████| 215.7MB 124kB/s 
     |████████████████████████████████| 204kB 41.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=4a999c6cc72a1a66ad41a45342792d6a0470edce565d252288634fa9510363e3
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *


from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)


In [0]:
df = spark.read.json("/content/persons.json")

In [8]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- height: long (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)



In [9]:
df.show()

+---+------+-------+------+
|age|height|   name|   sex|
+---+------+-------+------+
| 14|   156|  Alice|female|
| 21|   181|    Bob|  male|
| 27|   176|Charlie|  male|
| 24|   167|    Eve|female|
| 19|   172|Frances|female|
| 31|   191| George|  male|
+---+------+-------+------+



In [0]:
persons = spark.read \
          .option('inferSchema',True) \
          .option('header',True) \
           .csv('/content/persons_header.csv')

In [11]:
persons.show()

+---+------+-------+------+
|age|height|   name|   sex|
+---+------+-------+------+
| 23|   156|  Alice|female|
| 21|   181|    Bob|  male|
| 27|   176|Charlie|  male|
| 24|   167|    Eve|female|
| 19|   172|Frances|female|
| 31|   191| George|female|
+---+------+-------+------+



In [12]:
persons.printSchema()


root
 |-- age: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)



In [16]:
result = persons.select(persons.name,persons.age)
result.show()

+-------+---+
|   name|age|
+-------+---+
|  Alice| 23|
|    Bob| 21|
|Charlie| 27|
|    Eve| 24|
|Frances| 19|
| George| 31|
+-------+---+



In [0]:
result = persons.select(persons.name,upper(persons.name))

In [19]:
result.show()

+-------+-----------+
|   name|upper(name)|
+-------+-----------+
|  Alice|      ALICE|
|    Bob|        BOB|
|Charlie|    CHARLIE|
|    Eve|        EVE|
|Frances|    FRANCES|
| George|     GEORGE|
+-------+-----------+



In [0]:
result_alias = persons.select(persons.name,upper(persons.name)).alias('names_new')

In [22]:
result_alias.show()

+-------+-----------+
|   name|upper(name)|
+-------+-----------+
|  Alice|      ALICE|
|    Bob|        BOB|
|Charlie|    CHARLIE|
|    Eve|        EVE|
|Frances|    FRANCES|
| George|     GEORGE|
+-------+-----------+



In [0]:
result_alias = persons.select(persons.name,(persons.age**2).alias('age_sqr'))

In [24]:
result_alias.show()

+-------+-------+
|   name|age_sqr|
+-------+-------+
|  Alice|  529.0|
|    Bob|  441.0|
|Charlie|  729.0|
|    Eve|  576.0|
|Frances|  361.0|
| George|  961.0|
+-------+-------+



In [25]:
result = persons.select(concat(when(persons.sex == 'male', "Mr ").otherwise("Mrs "), persons.name).alias("salutation"))
result.show()

+-----------+
| salutation|
+-----------+
|  Mrs Alice|
|     Mr Bob|
| Mr Charlie|
|    Mrs Eve|
|Mrs Frances|
| Mrs George|
+-----------+

